In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

In [ ]:
# Code for ETL operations on Country-GDP data
# Importing the required libraries
import pandas as pd
from datetime import datetime
import requests
import numpy as np
from bs4 import BeautifulSoup
import sqlite3
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + ' : ' + message + '\n')
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    df = pd.DataFrame(columns = table_attribs )
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    data_list = []
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {table_attribs[0]: col[1].get_text(strip=True),
                           table_attribs[1] : float(col[2].get_text(strip=True))}
            data_list.append(data_dict)

    df = pd.DataFrame(data_list,columns = table_attribs )
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    exchange_rateDF = pd.read_csv(csv_path)
    exchange_rate = exchange_rateDF.set_index('Currency').to_dict()['Rate']

    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name,sql_connection,if_exists ='replace', index= False)
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    query_output = pd.read_sql(query_statement,sql_connection)
    print(query_statement)
    print(query_output)

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''
log_file = "code_log.txt"
csv_path = "exchange_rate.csv"
output_path = "./Largest_banks_data.csv"
db_name = 'Banks.db'
table_name = 'Largest_banks'
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
Excolumns = ["Name", "MC_USD_Billion"]

log_progress("Preliminaries complete. Initiating ETL process")
extracted_df = extract(url,Excolumns)
print(extracted_df)
print("\n")

log_progress("Data extraction complete. Initiating Transformation process")
transformed_df = transform(extracted_df,csv_path)
print(transformed_df)
print("\n")

log_progress("Data transformation complete. Initiating Loading process")

load_to_csv(transformed_df, output_path)
log_progress("Data saved to CSV file")

sql_connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

load_to_db(transformed_df, sql_connection, table_name)
log_progress("Data loaded to Database as a table, Executing queries")

query_statement = f"SELECT * FROM {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)
log_progress("Process Complete")

sql_connection.close()
log_progress("Server Connection closed")



